In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import io
import cv2
import time
# from matplotlib.axes._axes import _log as matplotlib_axes_logger 
# matplotlib_axes_logger.setLevel('ERROR')

In [44]:
def lidar_to_2d_front_view(points,
                           v_res,
                           h_res,
                           v_fov,
                           val,
                           cmap="jet",
                           saveto=None,
                           y_fudge=0.0
                           ):
    

    # DUMMY PROOFING
    # assert len(v_fov) ==2, "v_fov must be list/tuple of length 2"
    # assert v_fov[0] <= 0, "first element in v_fov must be 0 or negative"
    # assert val in {"depth", "height", "reflectance"}
        # 'val must be one of {"depth", "height", "reflectance"}'


    x_lidar = points[:, 0]
    y_lidar = points[:, 1]
    z_lidar = points[:, 2]
    r_lidar = points[:, 3] # Reflectance
    
    l_lidar = points[:, 4] #label
    
    # Distance relative to origin when looked from top
    d_lidar = np.sqrt(x_lidar ** 2 + y_lidar ** 2)
    # Absolute distance relative to origin
    # d_lidar = np.sqrt(x_lidar ** 2 + y_lidar ** 2, z_lidar ** 2)

    v_fov_total = -v_fov[0] + v_fov[1]

    # Convert to Radians
    v_res_rad = v_res * (np.pi/180)
    h_res_rad = h_res * (np.pi/180)

    # PROJECT INTO IMAGE COORDINATES
    x_img = np.arctan2(-y_lidar, x_lidar)/ h_res_rad
    y_img = np.arctan2(z_lidar, d_lidar)/ v_res_rad
    

    label_remap = {
    11: 10, 17: 10, 18: 10, 19: 10, 15: 11, 151: 11, 152: 11, 13: 18, 
    12: 13, 14: 15, 141: 15, 142: 15, 69: 80, 694: 80, 61: 99, 60: 253, 
    424: 60, 425: 60, 426: 49, 427: 49, 428: 49, 429: 49, 30: 50, 31: 50, 
    32: 50, 33: 50, 34: 50, 21: 30, 24: 16, 23: 31, 22: 32, 421: 40, 431: 40, 
    423: 20, 46: 44, 441: 44, 442: 44, 422: 48, 642: 51, 62: 52, 68: 52, 
    63: 252, 67: 258, 65: 254, 641: 255, 643: 256, 912: 70, 92: 70, 921: 70, 
    93: 72, 41: 72, 411: 72, 412: 72, 911: 71, 691: 81, 692: 257, 693: 257, 
    66: 259, 42: 40, 43:40, 90:0, 91:70
}


    learning_map = {
    0: 0,     # "unlabeled"
    1: 0,     # "outlier" mapped to "unlabeled"
    10: 1,    # "car"
    11: 2,    # "bicycle"
    13: 5,    # "bus" mapped to "other-vehicle"
    15: 3,    # "motorcycle"
    16: 5,    # "on-rails" mapped to "other-vehicle"
    18: 4,    # "truck"
    20: 5,    # "other-vehicle"
    30: 6,    # "person"
    31: 7,    # "bicyclist"
    32: 8,    # "motorcyclist"
    40: 9,    # "road"
    44: 10,   # "parking"
    48: 11,   # "sidewalk"
    49: 12,   # "other-ground"
    50: 13,   # "building"
    51: 14,   # "fence"
    52: 0,    # "other-structure" mapped to "unlabeled"
    60: 9,    # "lane-marking" to "road"
    70: 15,   # "vegetation"
    71: 16,   # "trunk"
    72: 17,   # "terrain"
    80: 18,   # "pole"
    81: 19,   # "traffic-sign"
    99: 0,    # "other-object" to "unlabeled"
    252: 1,   # "moving-car" to "car"
    253: 7,   # "moving-bicyclist" to "bicyclist"
    254: 6,   # "moving-person" to "person"
    255: 8,   # "moving-motorcyclist" to "motorcyclist"
    256: 5,   # "moving-on-rails" mapped to "other-vehicle"
    257: 5,   # "moving-bus" mapped to "other-vehicle"
    258: 4,   # "moving-truck" to "truck"
    259: 5    # "moving-other-vehicle" to "other-vehicle"
}

    
    # SHIFT COORDINATES TO MAKE 0,0 THE MINIMUM
    x_min = -360.0 / h_res / 2  # Theoretical min x value based on sensor specs
    x_img -= x_min              # Shift
    x_max = 360.0 / h_res       # Theoretical max x value after shifting
    #print("x_img",x_img)
    #print("x_max",x_max)

    y_min = v_fov[0] / v_res    # theoretical min y value based on sensor specs
    y_img -= y_min              # Shift
    y_max = v_fov_total / v_res # Theoretical max x value after shifting

    y_max += y_fudge            # Fudge factor if the calculations based on
                                # spec sheet do not match the range of
                                # angles collected by in the data.
                                
    pixel_values1=[] # save the colormap for each label value in list

   # Remap labels in l_lidar
    remapped_l_lidar = [label_remap[label] if label in label_remap else label for label in l_lidar]


    # WHAT DATA TO USE TO ENCODE THE VALUE FOR EACH PIXEL
    if val == "intensity":
        pixel_values = r_lidar
    elif val == "height":
        pixel_values = z_lidar

    elif val == 'label':
 
        
        for i in range(0, len(remapped_l_lidar)):
            # Access elements from remapped_l_lidar instead of l_lidar
            if remapped_l_lidar[i] in learning_map.keys():
                pixel_values1.append(learning_map[remapped_l_lidar[i]])
            else:
                # If the remapped label is not in color_map, append 'black'
                pixel_values1.append(0)

    
    else:
        pixel_values = d_lidar
        

    pixel_values=np.array(pixel_values1)
    #print(pixel_values.shape)

    # Rescale x_img and y_img to fit within 64x1024 grid
    x_img_scaled = (x_img / x_max) * 1024
    y_img_scaled = (y_img / y_max) * 64


    # Create a 2D array to store pixel values
    image_array = np.zeros((64, 1024))

    # Assign pixel values to the corresponding grid points
    for i in range(len(x_img_scaled)):
        x_index = int(round(x_img_scaled[i]))
        y_index = int(round(y_img_scaled[i]))
        if 0 <= x_index < 1024 and 0 <= y_index < 64:
            image_array[63-y_index, x_index] = pixel_values[i]

    if saveto is not None:
        np.save(saveto, image_array)

In [48]:
# set input and output folder paths
folder = 'Test'
input_folder = rf"/DATA2/Vivek/Vivekk/Data/VoxelScape/Csv_Files/{folder}"
output_folder = rf"/DATA2/Vivek/Vivekk/Data/VoxelScape/Voxel_Downstream_Task/{folder}/test_lidar_label"
label_folder_base = r"/DATA2/Vivek/Vivekk/Data/VoxelScape/Binary"
# create output folder if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [49]:
#For folder Hierarchy  with sequence folders
HRES = 0.35         # horizontal resolution (assuming 20Hz setting)
VRES=0.4
VFOV = (-25, 2) # Field of view (-ve, +ve) along vertical axis
Y_FUDGE = 0         # y fudge factor for velodyne HDL 64E
# val must be one of {"depth", "height", "intensity", "label"}


# loop through all CSV files in input folder
value='label'
# create output folder if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Start time for estimation
start_time = time.time()
total_files = len([name for name in os.listdir(input_folder) if name.endswith('.csv')])
processed_files = 0

for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        # read CSV file into pandas dataframe
        csv_file_path = os.path.join(input_folder, filename)
        df = pd.read_csv(csv_file_path)

        # Extract the sequence number from the file name
        sequence = filename.split('_')[0]

        # Construct label folder path (inside 'orig_labels' in sequence folder)
        label_folder = os.path.join(label_folder_base, sequence, 'orig_labels')

        # Remove the sequence number prefix and add "xyz_iom_" to label filename
        label_filename = "xyz_iom_" + filename[len(sequence) + 1:].replace('.csv', '.label')
        label_file_path = os.path.join(label_folder, label_filename)
        if os.path.exists(label_file_path):
            label = np.fromfile(label_file_path, dtype=np.uint32)
            label = label.reshape(-1)

            if 'label' not in df.columns:
                df.insert(4, 'label', label)
                df.to_csv(csv_file_path)

        # convert dataframe to numpy array
        lidar = np.array(df[['x', 'y', 'z', 'intensity', 'label']])

        # apply lidar_to_2d_front_view function
        output_filename = filename.replace('.csv', '.npy')
        output_path = os.path.join(output_folder, output_filename)
        lidar_to_2d_front_view(lidar, v_res=VRES, h_res=HRES, v_fov=VFOV,
                               val=value, saveto=output_path,
                               y_fudge=Y_FUDGE)

        # Time estimation
        processed_files += 1
        elapsed_time = time.time() - start_time
        estimated_time_remaining = elapsed_time / processed_files * (total_files - processed_files)
        print(f"Processed {processed_files} out of {total_files}. Estimated time remaining: {estimated_time_remaining:.2f} seconds.")

Processed 1 out of 2002. Estimated time remaining: 1568.65 seconds.
Processed 2 out of 2002. Estimated time remaining: 1593.95 seconds.
Processed 3 out of 2002. Estimated time remaining: 1639.05 seconds.
Processed 4 out of 2002. Estimated time remaining: 1622.26 seconds.
Processed 5 out of 2002. Estimated time remaining: 1610.60 seconds.
Processed 6 out of 2002. Estimated time remaining: 1610.82 seconds.
Processed 7 out of 2002. Estimated time remaining: 1613.05 seconds.
Processed 8 out of 2002. Estimated time remaining: 1602.70 seconds.
Processed 9 out of 2002. Estimated time remaining: 1595.84 seconds.
Processed 10 out of 2002. Estimated time remaining: 1614.53 seconds.
Processed 11 out of 2002. Estimated time remaining: 1607.97 seconds.
Processed 12 out of 2002. Estimated time remaining: 1604.94 seconds.
Processed 13 out of 2002. Estimated time remaining: 1599.70 seconds.
Processed 14 out of 2002. Estimated time remaining: 1602.34 seconds.
Processed 15 out of 2002. Estimated time re